In [1]:
from peewee import *
from random import randint
from typing import Any, Callable, TypeVar
import json

In [2]:
T = TypeVar('T')

In [3]:
def pad_start(width, fillchar, number) -> int:
    return int(str(number).rjust(width, fillchar))

def pad_end(width, fillchar, number) -> int:
    return int(str(number).ljust(width, fillchar))

In [4]:

def get_random_integer(min_value: T = 1, max_value: T = 9) -> int:
    min = pad_start(7,'1', pad_end(6, '0', min_value))
    max = pad_start(8, '9', pad_end(7, '9', max_value))
    return randint(min, max)

rand_number = get_random_integer()

print(rand_number)

24744309


In [5]:
database_name = 'notebooks_3_{}.db'.format(get_random_integer())
print(database_name)

notebooks_3_57870875.db


In [6]:
# db = SqliteDatabase(database_name, pragmas={'journal_mode':'wal','foreing_keys':1})
db = SqliteDatabase(database_name, pragmas={'foreing_keys':1})

In [7]:
# dir(db)

In [8]:
db.connection();

In [9]:
class BaseModel(Model):
    class Meta:
        database = db

In [10]:
class Person(BaseModel):
    id = AutoField()
    first_name = CharField()
    last_name = CharField()
    phone = CharField(null=True)
    
class Experience(BaseModel):
    id = AutoField()
    title = CharField()
    company = CharField()
    start_date = DateField()
    end_date = DateField(null=True)
    person = ForeignKeyField(Person, backref='experiences')
    
class Contact(BaseModel):
    id = AutoField()
    nickname = CharField()
    owner = ForeignKeyField(Person, backref='contacts')

class Phone(BaseModel):
    id = AutoField()
    type = CharField()
    number = CharField()
    contact = ForeignKeyField(Contact, backref='phones')


In [11]:
db.create_tables([Person, Experience, Contact, Phone])

In [12]:
data = {
    'person_record': {
        'first_name': 'Gabriel',
        'last_name': 'Ariza',
        'phone': '1234567890'
    },
    'experience_records': [
        {
            'title'      :      'Developer',
            'company'    :      'Google',
            'start_date' :      '2018-01-01',
            'end_date'   :      '2018-02-01',
            'person_id'     :      0
        }
    ],
    'contact_records': [
        {
            'nickname': 'john_doe',
            'owner_id': 0
        }
    ],

    'phone_records': [
        {
            'type': 'home',
            'number': '1234567890',
            'contact_id': 0
        }
    ]
}


In [13]:
insert_data = data.get('person_record', {})
print(json.dumps([insert_data], indent=4))

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890"
    }
]


In [14]:
insert_response = Person.create(**insert_data)
print(json.dumps([insert_response.__data__], indent=4))
# dir(insert_response)

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890",
        "id": 1
    }
]


In [15]:
insert_data = data.get('person_record', {})
insert_data['experiences'] = data.get('experience_records', [])
insert_data['contacts'] = data.get('contact_records', [])
for i, contact in enumerate(insert_data['contacts']):
    contact.update({'nickname': '{}. {}'.format((i + 1), contact.get('nickname', ''))})
    contact['phones'] = data.get('phone_records', [])

print(json.dumps([insert_data], indent=4))

[
    {
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890",
        "experiences": [
            {
                "title": "Developer",
                "company": "Google",
                "start_date": "2018-01-01",
                "end_date": "2018-02-01",
                "person_id": 0
            }
        ],
        "contacts": [
            {
                "nickname": "1. john_doe",
                "owner_id": 0,
                "phones": [
                    {
                        "type": "home",
                        "number": "1234567890",
                        "contact_id": 0
                    }
                ]
            }
        ]
    }
]


In [16]:
# The create method should insert the person record into the database
insert_response = Person.create(**insert_data)

In [17]:
print(type(insert_response)) # Should to be the class model: Person <Model: Person>
print(insert_response) # Should to output the person primary key (id) value (2)
print(insert_response.id) # Should be the same value as above as the both are outputting the primary key (id) value (2)

<Model: Person>
2
2


In [18]:
print(json.dumps([insert_response.__data__], indent=2))

[
  {
    "first_name": "Gabriel",
    "last_name": "Ariza",
    "phone": "1234567890",
    "id": 2
  }
]


In [19]:
print(json.dumps([insert_response.experiences], indent=2))

[
  [
    {
      "title": "Developer",
      "company": "Google",
      "start_date": "2018-01-01",
      "end_date": "2018-02-01",
      "person_id": 0
    }
  ]
]


In [20]:
for exp in insert_response.experiences:
    exp.update({'person_id': insert_response.id})

In [21]:
print(json.dumps(insert_response.experiences, indent=2))

[
  {
    "title": "Developer",
    "company": "Google",
    "start_date": "2018-01-01",
    "end_date": "2018-02-01",
    "person_id": 2
  }
]


In [22]:
print(json.dumps(insert_response.contacts, indent=4))

[
    {
        "nickname": "1. john_doe",
        "owner_id": 0,
        "phones": [
            {
                "type": "home",
                "number": "1234567890",
                "contact_id": 0
            }
        ]
    }
]


In [23]:
for contact in insert_response.contacts:
    contact.update({'owner_id': insert_response.id})

In [24]:
print(json.dumps(insert_response.contacts, indent=4))

[
    {
        "nickname": "1. john_doe",
        "owner_id": 2,
        "phones": [
            {
                "type": "home",
                "number": "1234567890",
                "contact_id": 0
            }
        ]
    }
]


In [25]:
experiencesEntity: list[Experience] = []
for index, exp in enumerate(insert_response.experiences):
    expEntity = Experience(**exp)
    expEntity.save()
    experiencesEntity.append(expEntity)

In [26]:
print(type(experiencesEntity[0]))

<Model: Experience>


In [27]:
print(type(experiencesEntity[0].person))

<Model: Person>


In [28]:
print(json.dumps([experiencesEntity[0].person.__data__], indent=4))

[
    {
        "id": 2,
        "first_name": "Gabriel",
        "last_name": "Ariza",
        "phone": "1234567890"
    }
]


In [29]:
contactsEntity: list[Contact] = []
for index, contact in enumerate(insert_response.contacts):
    contactEntity = Contact(**contact)
    contactEntity.save()
    contactsEntity.append(contactEntity)

In [30]:
lent_contactsEntity = len(contactsEntity)

In [31]:
print(lent_contactsEntity)

1


In [32]:
print(type(contactsEntity[0]) if lent_contactsEntity >= 1 else 'no contacts')

<Model: Contact>


In [33]:
print(type(contactsEntity[0].owner) if lent_contactsEntity >= 1 else 'no contact to access his owner')

<Model: Person>


In [34]:
print(type(contactsEntity[0].phones) if lent_contactsEntity >= 1 else 'no contact to access his phones')

<class 'list'>


In [35]:
print(json.dumps(insert_response.contacts, indent=2))

[
  {
    "nickname": "1. john_doe",
    "owner_id": 2,
    "phones": [
      {
        "type": "home",
        "number": "1234567890",
        "contact_id": 0
      }
    ]
  }
]


In [36]:
print(json.dumps(contactsEntity[0].phones, indent=2))

[
  {
    "type": "home",
    "number": "1234567890",
    "contact_id": 0
  }
]


In [37]:
print(contactsEntity[0].id)

1


In [38]:

for c in contactsEntity:
    phone_entity_list:list[Phone] = []
    for p in c.phones:
        p.update({'contact_id': c.id})
        phoneEntity = Phone(**p)
        phoneEntity.save()
        phone_entity_list.append(phoneEntity)
    # update the contact phones with the phone_entity_list
    # c.update({'phones': phone_entity_list})
    c.phones = phone_entity_list
    

In [39]:
phones = []
for c in contactsEntity:
    for p in c.phones:
        phones.append(p.__data__)

print(json.dumps(phones, indent=2))

[
  {
    "type": "home",
    "number": "1234567890",
    "contact": 1,
    "id": 1
  }
]


In [40]:
contactsEntity[0].phones[0].__data__

{'type': 'home', 'number': '1234567890', 'contact': 1, 'id': 1}

In [41]:
contactsEntity[0].phones[0].contact.__data__

{'id': 1, 'nickname': '1. john_doe', 'owner': 2}

In [42]:
contactsEntity[0].phones[0].contact.owner.__data__

{'id': 2, 'first_name': 'Gabriel', 'last_name': 'Ariza', 'phone': '1234567890'}

In [43]:
contactsEntity[0].phones[0].contact.owner.experiences[0].__data__

{'id': 1,
 'title': 'Developer',
 'company': 'Google',
 'start_date': datetime.date(2018, 1, 1),
 'end_date': datetime.date(2018, 2, 1),
 'person': 2}

In [44]:
phone_contact_owner_experience_list = []
for phone_contact_owner_experience in contactsEntity[0].phones[0].contact.owner.experiences:
    phone_contact_owner_experience.__data__.update({'start_date': phone_contact_owner_experience.__data__['start_date'].__str__()})
    phone_contact_owner_experience.__data__.update({'end_date': phone_contact_owner_experience.__data__['end_date'].__str__()})
    phone_contact_owner_experience_list.append(phone_contact_owner_experience.__data__)
# print(json.dumps(phone_contact_owner_experience_list, indent=2))
print(json.dumps(phone_contact_owner_experience_list, indent=2))

[
  {
    "id": 1,
    "title": "Developer",
    "company": "Google",
    "start_date": "2018-01-01",
    "end_date": "2018-02-01",
    "person": 2
  }
]


In [45]:
insert_response.experiences = experiencesEntity
insert_response.contacts = contactsEntity

In [46]:
print(insert_response.experiences[0].person.first_name)
print(insert_response.contacts[0].owner.first_name)

Gabriel
Gabriel


In [47]:

try:
    insert_response.__data__['experiences'] = insert_response.experiences
    insert_response.__data__['contacts'] = insert_response.contacts
except Exception as e:
    print(e)



In [48]:
print([insert_response])

[<Person: 2>]


In [49]:
print(insert_response.experiences)

[<Experience: 1>]


In [50]:
print(insert_response.contacts)

[<Contact: 1>]


In [51]:
print(insert_response.contacts[0].phones)

[<Phone: 1>]


In [52]:
for k,v in insert_response.__data__.items():
    print(k,': ',v)

first_name :  Gabriel
last_name :  Ariza
phone :  1234567890
id :  2
experiences :  [<Experience: 1>]
contacts :  [<Contact: 1>]


In [53]:
experiences = Experience().select()

In [54]:
print([exp for exp in experiences])

[<Experience: 1>]


In [55]:
contacts = Contact.select() # The person contacts should be retrived

In [56]:
print([c for c in contacts])

[<Contact: 1>]


In [57]:
print(len(insert_response.contacts[0].phones))
print(len(contacts[0].phones))

1
1


In [58]:
print(insert_response.contacts[0].nickname)
print(insert_response.contacts[0].phones[0].number)

1. john_doe
1234567890


In [59]:
db.close()

True